In [11]:
import numpy as np
import numba as nb

import ice_funcs as icef

In [12]:
icef.initialize_sat_map(1001,501)

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]])

## Todo:

- Finish SaturationRelaxationUtilities
- ***ISOLATE AND IRON OUT ALL KINKS IN ICE_FUNCS (COMMENT METHOD)***
- MAKE SURE THAT NEW CLASSES ARENT FUCKED UP
- make sure everything is numbafied
- Figure out how to diffuse_to_convergence
- Appliquer la diffusion
- Workout comment intégrer le filling field 'f'
- Continue commenting in `ice_funcs.py`

In [13]:
# CONSTANTS

L = 101
W = (L+1)//2
INITIAL_SAT = 1

In [14]:
# intialize instance of GeneralUtilities
gu = icef.GeneralUtilities(1001)

ice_map_test = icef.construct_minimal_ice_map(1001, 501)
ice_map_test[1001-4,0] = True
print(ice_map_test[-7:,:7])

[[False False False False False False False]
 [False False False False False False False]
 [False False False False False False False]
 [ True False False False False False False]
 [ True  True False False False False False]
 [ True False False False False False False]
 [ True False False False False False False]]


In [15]:
A = gu.construct_boundary_map(ice_map_test)[-7:, :7]
print(A)


[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [1 1 0 0 0 0 0]
 [0 3 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]
